In [15]:
import warnings
warnings.filterwarnings("ignore")
import os
import re
import time

import requests
from bs4 import BeautifulSoup, Comment
from requests.exceptions import ConnectionError
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd

By.PARTIAL_LINK_TEXT：定位

使用 ActionChains 不會立即執行

move_to_element：鼠標移動到某個元素

key_down：按下某個鍵盤的鍵

Keys.CONTROL：ctrl鍵

key_up：鬆開某個鍵盤的鍵


In [2]:
# 解析 HTML 內容
def get_bs4_content(url):
    res = requests.get(url, verify=False)
    soup = BeautifulSoup(res.text, "html.parser")
    return soup

In [3]:
# 將這些標籤文字內容 text 以逗號全部連結起來，就是完整的判決書內容了。
def get_full_text(content):
    nodes = content.find("body").find_all("td")
    full_text = ",".join([node.text for node in nodes]) # node.text：每一個 td 的文字
    return full_text

In [8]:
page_content

<html><head><title>Request Rejected</title></head><body>The requested URL was rejected. Please consult with your administrator.<br/><br/>Your support ID is: &lt; 6623888952434921945&gt;<br/><br/><a href="javascript:history.back();">[Go Back]</a></body></html>

In [23]:
name = "臺灣臺南地方法院"
period = [
    (111, 3, 1, 111, 3, 31),
    (111, 2, 1, 111, 2, 28),
    (111, 1, 1, 111, 1, 31),
    (110, 12, 1, 110, 12, 31),
    (110, 11, 1, 110, 11, 30),
]

article_data = pd.DataFrame()
article_id = 0

for y1, m1, d1, y2, m2, d2 in period:
    ops=Options()
    driver = webdriver.Chrome(ChromeDriverManager().install(),options=ops)
    url = "https://judgment.judicial.gov.tw/FJUD/Default_AD.aspx"
    driver.get(url)

    # 案件類別
    # a = driver.find_element(By.ID, "vtype_V")
    # a.find_element(By.NAME, "jud_sys").click()
    
    # 搜尋日期
    
    content = driver.find_element(By.ID, 'dy1') # 起始年
    content.send_keys(y1)
    time.sleep(10)

    content = driver.find_element(By.ID, "dm1") # 起始月
    content.send_keys(m1)

    content = driver.find_element(By.ID, "dd1") # 起始日
    content.send_keys(d1)

    content = driver.find_element(By.ID, "dy2") # 迄止年
    content.send_keys(y2)

    content = driver.find_element(By.ID, "dm2") # 迄止月
    content.send_keys(m2)

    content = driver.find_element(By.ID, "dd2") # 迄止日
    content.send_keys(d2)

    # 全文內容
    content = driver.find_element(By.NAME, "jud_kw")
    content.send_keys("交通事故")


    # 送出查詢
    driver.find_element(By.NAME, "ctl00$cp_content$btnQry").click()

    # 裁判法院，查詢結果會開啟於新分頁
    court = driver.find_element(By.PARTIAL_LINK_TEXT , name)
    actions = ActionChains(driver).move_to_element(court).key_down(Keys.CONTROL).key_down(
            Keys.SHIFT).click(court).key_up(Keys.CONTROL).key_up(Keys.SHIFT)
    actions.perform()

    # 轉換到新開的畫面
    handles = driver.window_handles
    driver.switch_to.window(handles[-1])
    time.sleep(0.5)

    page_url = driver.current_url
    page_num = 1 # 第一頁開始
    while True: 
        if page_num <= int(25):
            page_content = get_bs4_content(url=page_url)

            # 將查詢結果列表中所有判決的 url 預存下來
            article_urls = [
                    f'https://law.judicial.gov.tw/FJUD/{node.get("href")}'
                    for node in page_content.body.table.find_all("a", {"id": "hlTitle"})
                ]

            for article_url in article_urls:
                article_id  += 1
                content = get_bs4_content(url=article_url)
                full_text = get_full_text(content=content)
                row = pd.DataFrame({
                    "id": article_id,
                    "url": article_url,
                    "full_text": full_text
                }, index=[0]) # index 設成 0
                article_data = article_data.append(row, ignore_index=True)
                time.sleep(0.5)

            try:
                # get next_page_url and assign to page_url
                next_page_qurl = page_content.find(
                    "a", {"class": "page", "id": "hlNext"}).get("href")
                page_url = f'https://law.judicial.gov.tw{next_page_qurl} & ot = in'
            except AttributeError:
                break

            page_num += 1
        else:
            break
    time.sleep(180)

In [25]:
article_data.to_csv(name + "判決書_2.csv", index=False, encoding='utf-8')

In [24]:
article_data

,id,url,full_text
0,1,https://law.judicial.gov.tw/FJUD/data.aspx?ty=...,\n臺灣臺南地方法院刑事簡易判決111年度交簡字第1147號聲 請 人 臺灣臺南地方檢...
1,2,https://law.judicial.gov.tw/FJUD/data.aspx?ty=...,\n臺灣臺南地方法院刑事簡易判決111年度交簡字第1047號聲 請 人 臺灣臺南地方檢...
2,3,https://law.judicial.gov.tw/FJUD/data.aspx?ty=...,\n臺灣臺南地方法院刑事簡易判決111年度交簡字第1174號聲 請 人 臺灣臺南地方檢...
3,4,https://law.judicial.gov.tw/FJUD/data.aspx?ty=...,\n臺灣臺南地方法院刑事簡易判決111年度交簡字第1139號聲 請 人 臺灣臺南地方檢...
4,5,https://law.judicial.gov.tw/FJUD/data.aspx?ty=...,\n臺灣臺南地方法院刑事簡易判決111年度交簡字第1077號公 訴 人 臺灣臺南地方檢...
...,...,...,...
1235,1236,https://law.judicial.gov.tw/FJUD/data.aspx?ty=...,\n臺灣臺南地方法院刑事簡易判決110年度交簡字第3214號聲 請 人 臺灣臺南地方檢...
1236,1237,https://law.judicial.gov.tw/FJUD/data.aspx?ty=...,\n臺灣臺南地方法院刑事簡易判決110年度交簡字第3208號聲 請 人 臺灣臺南地方檢...
1237,1238,https://law.judicial.gov.tw/FJUD/data.aspx?ty=...,\n臺灣臺南地方法院刑事簡易判決110年度交簡字第3147號聲 請 人 臺灣臺南地方檢...
1238,1239,https://law.judicial.gov.tw/FJUD/data.aspx?ty=...,\n臺灣臺南地方法院刑事判決110年度交易字第783號公 訴 人 臺灣臺南地方檢察署檢...


NameError: name 'dy1' is not defined